In [4]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

def cust_lemma(tokens):
    for i in tokens:
        if len(i)<3 and i.isdigit()==False:
            tokens.remove(i)
        for j in corpus:
            if i in corpus[j]:
                tokens[tokens.index(i)] = j
 

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)
corpus = json.loads(open('corpus.json').read())


for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]

#remove two letter words
cust_lemma(words)
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=300, batch_size=5, verbose=1)
model.save('model.h5', hist)

print("model created")


88 documents
16 classes ['Abdurrahman', 'Abu Bakr 1', 'Abu Bakr 2', 'General', 'Umar 1', 'Umar 2', 'Zubair', 'ali', 'ali 2', 'juma', 'maghfirat', 'music', 'sad waqqaas', 'saeed zaid', 'ubaydah jarrah talhah ubaydullah', 'uthman']
69 unique lemmatized words ['1', '10', '11', '12', '13', '2', '3', '4', '5', '6', '7', '8', '9', 'abdurrahman', 'abu', 'ali', 'allah', 'asbaab', 'ashra', 'azeem', 'bakr', 'bhoolein', 'bhoolo', 'biography', 'din', 'dua', 'dusra', 'first', 'forgiveness', 'haram', 'hayaat', 'hyaat', 'ijtemai', 'jarrah', 'juma', 'karna', 'key', 'kirdaar', 'lamha', 'life', 'maafi', 'maango', 'mubashira', 'music', 'part', 'pehla', 'pucho', 'qabooliyat', 'rabb', 'rub', 'saat', 'sad', 'saeed', 'second', 'seerah', 'seerat', 'taalib', 'tadbeer', 'talhah', 'ubaydah', 'ubaydullah', 'umar', 'uthman', 'video', 'waqqaas', 'waqt', 'zayd', 'zindagi', 'zubair']
Training data created


C:\Users\multi\Anaconda3\envs\chatbot\lib\site-packages\ipykernel_launcher.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Epoch 1/300
88/88 [==============================] - 1s 9ms/step - loss: 2.7989 - accuracy: 0.1136
Epoch 2/300
88/88 [==============================] - 0s 608us/step - loss: 2.7059 - accuracy: 0.0795
Epoch 3/300
88/88 [==============================] - 0s 355us/step - loss: 2.6392 - accuracy: 0.1364
Epoch 4/300
88/88 [==============================] - 0s 429us/step - loss: 2.5523 - accuracy: 0.1591
Epoch 5/300
88/88 [==============================] - 0s 355us/step - loss: 2.3930 - accuracy: 0.2045
Epoch 6/300
88/88 [==============================] - 0s 355us/step - loss: 2.2858 - accuracy: 0.2841
Epoch 7/300
88/88 [==============================] - 0s 429us/step - loss: 2.1065 - accuracy: 0.3523
Epoch 8/300
88/88 [==============================] - 0s 533us/step - loss: 2.0033 - accuracy: 0.3750
Epoch 9/300
88/88 [==============================] - 0s 429us/step - loss: 1.9576 - accuracy: 0.4886
Epoch 10/300
88/88 [==============================] - 0s 355us/step - loss: 1.7949 - accuracy

88/88 [==============================] - 0s 430us/step - loss: 0.1341 - accuracy: 0.9659
Epoch 82/300
88/88 [==============================] - 0s 355us/step - loss: 0.1768 - accuracy: 0.9318
Epoch 83/300
88/88 [==============================] - 0s 533us/step - loss: 0.1733 - accuracy: 0.9545
Epoch 84/300
88/88 [==============================] - 0s 356us/step - loss: 0.1631 - accuracy: 0.9545
Epoch 85/300
88/88 [==============================] - 0s 355us/step - loss: 0.1404 - accuracy: 0.9659
Epoch 86/300
88/88 [==============================] - 0s 429us/step - loss: 0.1143 - accuracy: 0.9773
Epoch 87/300
88/88 [==============================] - 0s 355us/step - loss: 0.1563 - accuracy: 0.9432
Epoch 88/300
88/88 [==============================] - 0s 178us/step - loss: 0.1234 - accuracy: 0.9432
Epoch 89/300
88/88 [==============================] - 0s 252us/step - loss: 0.1091 - accuracy: 0.9773
Epoch 90/300
88/88 [==============================] - 0s 355us/step - loss: 0.1663 - accuracy: 

88/88 [==============================] - 0s 355us/step - loss: 0.0638 - accuracy: 0.9886
Epoch 161/300
88/88 [==============================] - 0s 533us/step - loss: 0.0350 - accuracy: 1.0000
Epoch 162/300
88/88 [==============================] - 0s 608us/step - loss: 0.0999 - accuracy: 0.9773
Epoch 163/300
88/88 [==============================] - 0s 533us/step - loss: 0.0533 - accuracy: 0.9886
Epoch 164/300
88/88 [==============================] - 0s 430us/step - loss: 0.0432 - accuracy: 1.0000
Epoch 165/300
88/88 [==============================] - 0s 355us/step - loss: 0.1671 - accuracy: 0.9318
Epoch 166/300
88/88 [==============================] - 0s 435us/step - loss: 0.0907 - accuracy: 0.9659
Epoch 167/300
88/88 [==============================] - 0s 177us/step - loss: 0.0870 - accuracy: 0.9886
Epoch 168/300
88/88 [==============================] - 0s 355us/step - loss: 0.0348 - accuracy: 1.0000
Epoch 169/300
88/88 [==============================] - 0s 252us/step - loss: 0.0414 - a

88/88 [==============================] - 0s 355us/step - loss: 0.0568 - accuracy: 0.9886
Epoch 240/300
88/88 [==============================] - 0s 355us/step - loss: 0.0339 - accuracy: 1.0000
Epoch 241/300
88/88 [==============================] - 0s 178us/step - loss: 0.0325 - accuracy: 0.9773
Epoch 242/300
88/88 [==============================] - 0s 355us/step - loss: 0.0220 - accuracy: 1.0000
Epoch 243/300
88/88 [==============================] - 0s 355us/step - loss: 0.1299 - accuracy: 0.9432
Epoch 244/300
88/88 [==============================] - 0s 252us/step - loss: 0.0205 - accuracy: 1.0000
Epoch 245/300
88/88 [==============================] - 0s 355us/step - loss: 0.0433 - accuracy: 0.9886
Epoch 246/300
88/88 [==============================] - 0s 178us/step - loss: 0.0680 - accuracy: 0.9659
Epoch 247/300
88/88 [==============================] - 0s 252us/step - loss: 0.0239 - accuracy: 1.0000
Epoch 248/300
88/88 [==============================] - 0s 355us/step - loss: 0.0359 - a